**The code below to create the 4x4 mapping should create a positive mapping**

In [ ]:
# This code was created by Darshini Rajamani of Purdue University

import random;

def get_random_number():
    return random.randint(-9, 9)

# Setting the requirements for PSD matrices
def is_valid(matrix, rows, cols, row, col, num):
    # initial assigment to check validation
    matrix[row][col] = num

    # Check row condition (check in E(2,2))
    # sum of the first two equals the sum of the last two
    if col == 3:
        if sum(matrix[row][:2]) != matrix[row][2] + num:
            return False

    # Check column condition (linearity check)
    # sum of the 1st and 2nd rows equal the 3rd and 4th
    if row == 3:
        if matrix[0][col] + matrix[1][col] != matrix[2][col] + num:
            return False

    # Positivity check
    # Sum of any element added to any other element is positive
    if col == 3:
        matrix[row][col] = num
        if matrix[row][0] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][0] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False

    matrix[row][col] = None
    return True

# Creating the matrices
def create_matrix(rows, cols):
    matrix = [[None for _ in range(cols)] for _ in range(rows)] #has every element in the matrix as 'none'

    backtrackFailedAttempts = 0
    def backtrack(row, col):  # to check for complete and valid solution
        nonlocal backtrackFailedAttempts
        if backtrackFailedAttempts > 1000:
            return False

        if row == rows:
            return True

        nums = [i for i in range(-9, 10)]  # range [-9, 10)
        random.shuffle(nums)

        for num in nums:
            if is_valid(matrix, rows, cols, row, col, num):
                matrix[row][col] = num

                next_row = row
                next_col = col + 1
                if next_col == cols:
                    next_row += 1
                    next_col = 0

                if backtrack(next_row, next_col):
                    return True

        matrix[row][col] = None
        backtrackFailedAttempts = backtrackFailedAttempts + 1
        return False

    backtrack(0, 0)  # recursion

    return matrix

In [ ]:
# this code was created by Luke Luschwitz (mainly) and Karim with influence from Abbas and Darshini

from scipy.optimize import linprog
import numpy as np

numberOfMappingsToCreate = 12000
#######################
### Create Mappings ###
#######################
listOfMappings = []
for i in range(numberOfMappingsToCreate):
  rows = 4
  cols = 4

  isAValidMatrix = False

  while not isAValidMatrix:
    isAValidMatrix = True
    matrix = create_matrix(rows, cols)
    for row in matrix:
      for element in row:
        if element is None:
          isAValidMatrix = False

  # matrix[1], matrix[3] = matrix[3], matrix[1]

  listOfMappings.append(matrix)


###########################
# Check for Extendability #
###########################
extendableMappings = []
nonExtendableMappings = []

for matrix in listOfMappings:
  mat = [1, 1, 1, 1]
  for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]
  solve_for = [[min(matrix[0][2], matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]
  for_A_eq = [[1,1,-1,-1]]
  for_b_eq = [0]

  result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
  for row in matrix:
    print(row)
  if (result.success):
    extendableMappings.append(matrix)
    print("Extendable")
  else:
    nonExtendableMappings.append(matrix)
    print("Not extendable")
  print()

# Save the sample data to a file
np.save('/content/extendableMappings.npy', extendableMappings)
np.save('/content/nonExtendableMappings.npy', nonExtendableMappings)

In [1]:
import numpy as np
extendableMappings = np.load('/content/extendableMappings.npy')
nonExtendableMappings = np.load('/content/nonExtendableMappings.npy')

In [5]:
import numpy as np
extendableMappingsEXTRA = np.load('/content/extendableMappings (1).npy')
nonExtendableMappingsEXTRA = np.load('/content/nonExtendableMappings (1).npy')

In [6]:
extendableMappings = np.concatenate((extendableMappings, extendableMappingsEXTRA))
nonExtendableMappings = np.concatenate((nonExtendableMappings, nonExtendableMappingsEXTRA))

In [7]:
print(extendableMappings.shape)

(27842, 4, 4)


In [8]:
print(f"extendableMappings: {len(extendableMappings)}")
print(f"nonExtendableMappings: {len(nonExtendableMappings)}")

extendableMappings: 27842
nonExtendableMappings: 8158


In [ ]:
# All the code below was made by Luke with slight influence from Abbas

import numpy as np

# This function returns the matrix B that is farthest away from the matrices in setA
def farthestBFromA(setB, setA):
  totalDistancesFromA = []
  # iterate through each B matrix
  for B in setB:
    currentBTotalDistance = 0
    # accumulate sum of distances from A to B (distance squared)
    for A in setA:
      currentBTotalDistance = currentBTotalDistance + (np.linalg.norm(np.subtract(B, A)) ** 2)
    totalDistancesFromA.append(currentBTotalDistance)

  orderedBsByDistance = []
  for i in range(len(totalDistancesFromA)):
    index = np.argmax(totalDistancesFromA)
    orderedBsByDistance.append(setB[index])
    totalDistancesFromA[index] = -np.inf
  return orderedBsByDistance


farthestBs = farthestBFromA(nonExtendableMappings, extendableMappings)

for i in range(len(farthestBs[0:3])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

In [11]:
for i in range(len(farthestBs[0:10])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

[ 9 -3  3  3]
[ 2  5  9 -2]
[ 9 -3  3  3]
[ 2  5  9 -2]
0th farthest from the extendables

[ 5  3  9 -1]
[9 9 9 9]
[ 5  3  9 -1]
[9 9 9 9]
1th farthest from the extendables

[ 2  6  9 -1]
[9 9 9 9]
[ 2  6  9 -1]
[9 9 9 9]
2th farthest from the extendables

[9 0 9 0]
[ 6 -1  4  1]
[ 9 -2  5  2]
[ 6  1  8 -1]
3th farthest from the extendables

[ 9 -3  3  3]
[ 6  1 -1  8]
[7 0 0 7]
[ 8 -2  2  4]
4th farthest from the extendables

[ 3  3  9 -3]
[9 8 8 9]
[ 3  3  8 -2]
[9 8 9 8]
5th farthest from the extendables

[7 2 0 9]
[ 7  1 -1  9]
[9 0 0 9]
[ 5  3 -1  9]
6th farthest from the extendables

[9 0 9 0]
[ 5  2  8 -1]
[7 1 8 0]
[ 7  1  9 -1]
7th farthest from the extendables

[ 9 -2  4  3]
[ 7  1 -1  9]
[7 0 2 5]
[ 9 -1  1  7]
8th farthest from the extendables

[9 0 9 0]
[ 4 -1  1  2]
[ 9 -3  3  3]
[ 4  2  7 -1]
9th farthest from the extendables



In [22]:
# Save the farthestBs to a file
np.save('/content/farthestBs.npy', farthestBs)

In [ ]:
# Hard code a few nonextendable matrices that are far away from the extendable matrices
# Run this code block instead of the previous code block
farthestBs = [[[5, 3, 9, -1],
               [9, 9, 9, 9],
               [5, 3, 9, -1],
               [9, 9, 9, 9]],

              [[2, 6, 9, -1],
               [9, 9, 9, 9],
               [2, 6, 9, -1],
               [9, 9, 9, 9]],

              [[9, 0, 9, 0],
               [5, 2, 8, -1],
               [7, 1, 8, 0],
               [7, 1, 9, -1]]]

for i in range(len(farthestBs[0:3])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

In [12]:
# Find a classifier that linearly separates the extendable matrices from the nonextendable matrices using an SVM

import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score

# Ensure everything is a numpy array
extendableMappings = np.array(extendableMappings)
nonExtendableMappings = np.array(nonExtendableMappings)
farthestBs = np.array(farthestBs)

# Generate input by adding the farthestB matrix to the list of extendableMappings
features = np.concatenate((
    extendableMappings,
    [farthestBs[0]],
    [extendableMappings[0]/1000],
    [farthestBs[0]/1000],
    [np.zeros((4, 4))]
    ))
print(f"len(features): {len(features)}")

# Flatten the matrices into 1D arrays
features = features.reshape(len(features), -1)

# Generate labels (0 or 1)
labels = [0]*len(extendableMappings) + [1,0,1,1]
print(f"len(labels): {len(labels)}")
print(f"labels: {labels}")

# Create and train a support vector classifier
model = svm.SVC(kernel='linear', C=1e10, coef0=0.0, tol=1e-5)
model.fit(features, labels)

# Set the bias (intercept) to be zero
model.intercept_ = [0.0]

# Make predictions on the test set
predictions = model.predict(features)

# Evaluate the accuracy of the model
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

# Get the coefficients (weights) of the hyperplane
# coefficients = model.coef_.reshape(4,4)
coefficients = model.coef_

# Intercept of the hyperplane
intercept = model.intercept_

print("Coefficients:\n", coefficients) #the matrix that we get from the ML program
print("Intercept:", intercept)


len(features): 27846
len(labels): 27846
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [19]:
# Double checks that the classifier (hyperplane) separates data
# Tweaks the classifier slightly to find a 'nicer' classifier with whole numbers and low significant digits

print("coefficients")
print(coefficients.reshape(4,-1))

classifier = np.round(coefficients/10) # this is same as coefficients, but times 10 and rounded to integers
print("classifier")
print(classifier.reshape(4,-1))

print()

allMappings = np.concatenate((extendableMappings, nonExtendableMappings), axis=0)
allMappings = allMappings.reshape(len(allMappings), -1)
print()

totalMappingsInClass1 = 0
totalMappingsInClass2 = 0
classifiedMappingResults = []

for i in range(len(allMappings)):
  dotProduct = np.dot(classifier, allMappings[i])
  dotProduct = np.sum(dotProduct)
  extendableOrNot = "E" if i<len(extendableMappings) else "N"
  if dotProduct < 0:
    # print(f"{extendableOrNot} {dotProduct} (class 1)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 1))
    totalMappingsInClass1 += 1
  else:
    # print(f"{extendableOrNot} {dotProduct} (class 2)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 2))
    totalMappingsInClass2 += 1

for mapping, extendability, _class in classifiedMappingResults:
  if (extendability == "E" and _class == 2):
    print("Bad classifier, extendableMapping in class 2")
    # print(mapping, extendability, _class)
    break
  if totalMappingsInClass1 < 1:
    print("Bad classifier, no mappings in class 1")
    break
  if totalMappingsInClass2 < 1:
    print("Bad classifier, no mappings in class 2")
    break

print(f"totalMappingsInClass1 {totalMappingsInClass1}")
print(f"totalMappingsInClass2 {totalMappingsInClass2}")


coefficients
[[  9.07184352 -52.0808405  -19.84148293 -23.16751406]
 [-50.40549311   2.10578759   9.17828652 -57.47799204]
 [ -5.7979263  -10.58234206  -8.08252413  -8.29774423]
 [-35.53572328 -39.39271085  -2.58067227 -72.34776186]]
classifier
[[ 1. -5. -2. -2.]
 [-5.  0.  1. -6.]
 [-1. -1. -1. -1.]
 [-4. -4. -0. -7.]]


totalMappingsInClass1 35998
totalMappingsInClass2 2


In [20]:
from scipy.optimize import linprog

c = -classifier.reshape(1, 16)

for_A_ub = [[-1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [-1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0], # positivity check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1]] # positivity check
for_b_ub = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for_A_eq = [[1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0], # linearity check
            [0, 1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0], # linearity check
            [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0], # linearity check
            [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1], # linearity check
            [1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # E(2,2) check
            [0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0], # E(2,2) check
            [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0], # E(2,2) check
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1]] # E(2,2) check
for_b_eq = [0, 0, 0, 0, 0, 0, 0, 0]

result = linprog(c=c, A_ub = for_A_ub, b_ub = for_b_ub, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)

print(result)

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [-0.000e+00  0.000e+00 ... -0.000e+00  0.000e+00]
           nit: 4
         lower:  residual: [-0.000e+00  0.000e+00 ... -0.000e+00  0.000e+00]
                marginals: [ 0.000e+00  7.000e+00 ...  0.000e+00  7.000e+00]
         upper:  residual: [       inf        inf ...        inf        inf]
                marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
         eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                             0.000e+00  0.000e+00  0.000e+00  0.000e+00]
                marginals: [-0.000e+00 -1.000e+00 -0.000e+00 -0.000e+00
                            -1.000e+00  1.000e+00 -0.000e+00 -0.000e+00]
       ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                marginals: [-0.000e+00 -0.000e+00 ... -0.000e+00 -0.000e+00]
